In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-udqehl4j
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-udqehl4j
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=3016150bb4599efcd50d54c9bb0ac85c53ef45e82cef23eaccc784292a9fed2a
  Stored in directory: /tmp/pip-ephem-wheel-cache-pr0h4fks/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [18]:
%%cu
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <cstring>
#include <cuda_runtime.h>

#define T 1000 // Maximum number of contacts
#define S 50   // Maximum string length for name and phone

using namespace std;

struct Contact {
    char name[S];
    char phone[S];
};

// Kernel
__global__ void searchContact(Contact* contacts, const char* searchName, int* result, int totalContacts) {
    int i = threadIdx.x;

    // Calculate the length of the searchName and contacts[i].name
    int searchNameLen = 0;
    for (int j = 0; searchName[j] != '\0'; j++)
        searchNameLen++;

    int contactLen = 0;
    for (int j = 0; contacts[i].name[j] != '\0'; j++)
        contactLen++;

    if (searchNameLen <= contactLen) {
        int strLen = searchNameLen;
        int flag = 1;

        for (int j = 0; j < strLen; ++j) {
            // Check for character-wise equality
            if (searchName[j] != contacts[i].name[j]) {
                flag = 0;
                break;
            }
        }

        // This part is for exact matching
        /*if (searchNameLen != contactLen) {
            flag = 0;
        }*/

        if (flag == 1) {
            // If match is found, store the index in result array
            result[i] = i;
        } else {
            // If no match is found, store -1 in result array
            result[i] = -1;
        }
    } else {
        // If searchName is longer than contacts[i].name, store -1 in result array
        result[i] = -1;
    }
}

int main() {
    // Host variables
    Contact contacts[T];
    char searchName[S] = "Joh"; // Default search name

    const char* phonebookFile_1 = "phonebook1.txt";
    const char* phonebookFile_2 = "phonebook2.txt";

    int totalContacts = 0;
    string line;

    // Read data from file_1
    ifstream file_1(phonebookFile_1);

    if (!file_1.is_open()) {
        cerr << "Error opening one or more files!" << endl;
        return 1;
    }

    // Parse each line and store in contacts array
    while (getline(file_1, line)) {
        istringstream iss(line);
        iss.getline(contacts[totalContacts].name, S, ',');  // extract characters from 'iss' until a comma (',') is encountered,
        iss.getline(contacts[totalContacts].phone, S);      // extract characters from 'iss' until a newline or end of stream is encountered
        totalContacts += 1;
    }
    file_1.close();

    // Read data from file_2
    ifstream file_2(phonebookFile_2);

    if (!file_2.is_open()) {
        cerr << "Error opening one or more files!" << endl;
        return 1;
    }
    // Parse each line and store in contacts array
    while (getline(file_2, line)) {
        istringstream iss(line);
        iss.getline(contacts[totalContacts].name, S, ',');
        iss.getline(contacts[totalContacts].phone, S);
        totalContacts += 1;
    }

    file_2.close();

    // CUDA event variables for timing
    cudaEvent_t start, end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    // Device memory
    Contact* dContacts;
    char* dSearchName;
    int* dResult;

    int t = totalContacts;

    // Allocate device memory
    cudaMalloc((void**)&dContacts, sizeof(Contact) * t);
    cudaMalloc((void**)&dSearchName, S * sizeof(char));
    cudaMalloc((void**)&dResult, sizeof(int) * t);

    // Copy data from host to device
    cudaMemcpy(dContacts, contacts, sizeof(Contact) * t, cudaMemcpyHostToDevice);
    cudaMemcpy(dSearchName, searchName, sizeof(char) * S, cudaMemcpyHostToDevice);

    // Set kernel launch parameters
    dim3 gridSize(1);   // Set the x, y, and z dimensions of the grid to 1.
    dim3 blockSize(t);

    // Launch kernel
    searchContact<<<gridSize, blockSize>>>(dContacts, dSearchName, dResult, totalContacts);
    cudaDeviceSynchronize();

    // Copy result back to host
    int result[t] = {0};
    cudaMemcpy(result, dResult, sizeof(int) * t, cudaMemcpyDeviceToHost);

    int notFound = 1;

    // Print results
    for (int y = 0; y < t; y++) {
        if (result[y] >= 0) {
            notFound = 0;
            cout << contacts[result[y]].name << "  " << contacts[result[y]].phone << endl;
        }
    }

    if (notFound) {
        cout << "Not Found" << endl;
    }

    // Record and print execution time
    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, end);
    cout << "\nTime taken: " << milliseconds << " milliseconds." << endl;

    // Free device memory
    cudaFree(dContacts);
    cudaFree(dResult);

    return 0;
}


John Doe  123-456-7890
John Smith  1234567890
John Lewis  0123456789
John Moris  123-456-7877

Time taken: 0.428832 milliseconds.

